In [30]:
import re

# 1. 基本匹配

对大小写敏感

In [31]:
print(re.findall(r"the", "The fat cat sat on the mat"))
print(re.findall(r"The", "The fat cat sat on the mat"))

['the']
['The']


简写字符集
|简写|描述|
|:----:|----|
|`\w`|匹配所有字母数字, 等同于 `[a-zA-Z0-9_]`|
|`\W`|匹配所有非字母数字, 即符号, 等同于: `[^\w]`|
|`\d`|匹配数字, 等同于: `[0-9]`|
|`\s`|匹配所有空格字符, 等同于: `[ \t\n\r\f\v]`|
|`\`|转义字符, 用于匹配一些保留的字符 `[ ] ( ) { } . * + ? ^ $ \`|
|`\b`|匹配一个位置, 前字符是 `\w`, 后字符是 `\W` (零宽匹配, 即本身不消耗字符, 只是一个位置检查)|

In [32]:
print(re.findall(r"\bthe\b", "the theatre is there"))

['the']


# 2. 元字符

## 2.1 点运算符 `.`

`.` 匹配任意非`\n`的单个字符.

In [33]:
print(re.findall(r".ar", "The car parked in the garage"))
print(re.findall(r".ar", "Start here, mark the area."))

['car', 'par', 'gar']
['tar', 'mar', ' ar']


## 2.2 字符集 `[]`

In [34]:
print(re.findall(r"[fcsm]at", "The fat cat sat on the mat"))
print(re.findall(r"[0-9]", "Today is 2025-08-10"))

['fat', 'cat', 'sat', 'mat']
['2', '0', '2', '5', '0', '8', '1', '0']


否定字符集

In [35]:
print(re.findall(r"[^c]ar", "The car parked in the garage"))
print(re.findall(r"[^a-z]g", "The car parked in the garage"))

['par', 'gar']
[' g']


## 2.3 重复字符 `*`, `+` (Greedy vs lazy matching)

`*` 号匹配 "在 `*` 之前的字符", <mark>出现 $\ge 0$ 次</mark>

`+` 号匹配 "在 `+` 之前的字符", <mark>出现 $\ge 1$ 次</mark>

In [36]:
print(re.findall(r"[a-z]*", "The car parked in the garage #21")) # 会匹配空字符
print(re.findall(r"[a-z]+", "The car parked in the garage #21")) # 不匹配空字符

['', 'he', '', 'car', '', 'parked', '', 'in', '', 'the', '', 'garage', '', '', '', '', '']
['he', 'car', 'parked', 'in', 'the', 'garage']


In [37]:
print(re.findall(r"\s*cat\s*", "The cat  sat on the concatenate"))

[' cat  ', 'cat']


`.*` 匹配 0 个或多个任意字符 (贪婪模式, 即 `*` 吃掉尽可能多的字符)

In [38]:
print(re.findall(r".*", "The car parked in the garage"))
print(re.findall(r"c.+t", "The fat cat sat on the mat"))

['The car parked in the garage', '']
['cat sat on the mat']


`*?` 和 `+?` 非贪婪模式, 即尽量少匹配字符

In [39]:
print(re.findall(r"g.*?t", "git is great for tracking changes and collaborating on code projects."))

['git', 'great', 'g changes and collaborat', 'g on code project']


## 2.4 长度符号 `{}`

`{n,m}` 匹配 num 个大括号之前的字符或字符集 (`n <= num <= m`) , `{n,}` 表示 `n <= num` , `{,m}` 表示 `num <= m` , `{n}` 表示 `num = n`


In [40]:
print(re.findall(r"\b\w{2,4}\b", "git is great for tracking changes and collaborating on code projects."))
print(re.findall(r"\w{4,}", "GitHub is great for sharing and collaborating on code projects.")) # 截取所有长度 \ge 4 的单词
print(re.findall(r"\b\d{1,4}\b", "64, 128, 1024, 65536"))

['git', 'is', 'for', 'and', 'on', 'code']
['GitHub', 'great', 'sharing', 'collaborating', 'code', 'projects']
['64', '128', '1024']


例如, markdown中用 `####` 表示四级标题

In [41]:
print(re.findall(r'^#{4}\s.*', "#### level 4"))
print(re.findall(r'^#{4}\s.*', "### level 3"))

['#### level 4']
[]


# 3. Python Re API

## 3.1 `()` 捕获组和 `(?:)`非捕获组

In [42]:
print(re.findall(r"(f|c|s| )at", "The fat cat sat at the door."))
print(re.findall(r"([a-z])at", "The fat cat sat on the mat."))

['f', 'c', 's', ' ']
['f', 'c', 's', 'm']


In [43]:
print(re.findall(r"(\d{3})-(\d{4})-(\d{4})", "I have two phone numbers: 130-3110-7085 and 186-1157-8994."))

[('130', '3110', '7085'), ('186', '1157', '8994')]


In [44]:
print(re.findall(r"(\d{4})-(?:\d{2})-(\d{2})", "2025-08-10")) # 其中月份为非捕获组

[('2025', '10')]


## 3.2 `re.sub(pattern, repl, string)`

用 `repl` 替换掉所以 `pattern` (`repl` 可以是一个函数)

In [45]:
md_line = "This is **important**"
# **text** -> \textbf{text}
tex_line = re.sub(r'\*\*(.*?)\*\*', r'\\textbf{\1}', md_line)
print(tex_line)

This is \textbf{important}


In [46]:
md_line = "Github : [yuanyi-350](https://github.com/yuanyi-350)"
# [label](url) -> \href{url}{label}
tex_line = re.sub(r'\[(.*?)\]\((.*?)\)', r'\\href{\2}{\1}', md_line)
print(tex_line)

Github : \href{https://github.com/yuanyi-350}{yuanyi-350}


`\1`, `\2` 分别表示替换为捕获组 1, 捕获组 2 的内容

## 3.3 `match` 对象

`re.match()` : 从字符串的<mark>起始位置</mark>开始匹配, 匹配成功返回一个 `match` 对象.

`re.search()` : 在整个字符串中搜索<mark>第一个匹配项</mark>, 匹配成功返回一个 `match` 对象

`.start(n)` 和 `.end(n)` : 分别返回捕获组 n 在原字符串的起始位置和终止位置

`.group(n)` : 返回第 n 个捕获组内容, `.group()` 和 `.group(0)` 表示整个匹配

`.groups()` : 返回一个元组，包含所有捕获组

In [47]:
m = re.search(r"(\d{3})-(\d{4})-(\d{4})", "I have two phone numbers: 130-3110-7085 and 186-1157-8994.")
print(m.group())  # 整个匹配
print(m.group(1)) # 第1组
print(m.group(2)) # 第2组
print(m.group(3)) # 第3组

130-3110-7085
130
3110
7085


In [48]:
m = re.match(r"(\d{4})-(?:\d{2})-(\d{2})", "2025-08-10 is the ddl") # 其中月份为非捕获组
print(m.groups())

('2025', '10')


In [49]:
m = re.search(r".at", "The fat cat sat on the mat")
if m:
    print(m.group(), m.start(), m.end())

fat 4 7


## 3.4 `re.compile()` 预编译

多次使用 `pattern` 模式匹配时可以预编译提升效率

In [50]:
equation_pattern = re.compile(r"(\$\$(.+?)\$\$)", re.DOTALL)

`re.DOTALL` 让 `.` 可以匹配包括换行符 `\n` 在内的所有字符.


## 3.5 锚点

`^` 匹配开头, `$` 匹配结尾. `re` 的 `flag` 若加入 `re.M` 将 `\n` 理解为换行, 若无则当成一个字符串整体

In [51]:
doc = """1. line1
2. line2
3. line3
""" # doc = "1. line1\n2. line2\n3. line3"
print(re.findall(r"^\d", doc, flags=re.M))
print(re.findall(r"^\d", doc))

['1', '2', '3']
['1']


# 4. 零宽断言

| 语法         | 名称      | 方向   | 作用            |
| ---- | ---- |------| ---- |
| `(?=...)`  | 正先行断言 | 向前   | 后面必须是 ... |
| `(?!...)`  | 负先行断言 | 向前   | 后面不能是 ... |
| `(?<=...)` | 正后发断言 | 向后   | 前面必须是 ... |
| `(?<!...)` | 负后发断言 | 向后   | 前面不能是 ... |

`X(?=Y)Z` Example :

先匹配 X, 然后从 X 后面开始与 Y 匹配, 最后从 X 后面开始与 Z 匹配 (Y 不出现在最终匹配文本里)

零宽: Y 匹配后, 匹配光标仍停在原地

In [52]:
print(re.findall(r"\w+(?=\.com)", "google.com, bilibili.com, zhihu.com"))

['google', 'bilibili', 'zhihu']


In [53]:
print(re.findall(r"(?=[^be])[a-z]", "abcdefgh"))

['a', 'c', 'd', 'f', 'g', 'h']


密码强度校验 (至少 8 位, 含大写, 小写, 数字)

In [54]:
print(re.findall(r"^(?=.*[A-Z])(?=.*[a-z])(?=.*\d).{8,}$", "Password0!*"))

['Password0!*']


1. `^` 匹配字符串的开头
2. `(?=.*[A-Z])` 正先行断言：从开头向后, 经过若干给字符 (包括 0 个), 可以匹配到大写字母
3. `(?=.*[a-z])` 存在小写字母
4. `(?=.*\d)`    存在数字
5. `.{8,}`       此时光标仍在第 0 位, 匹配任意字符, 长度至少 8
6. `$`           匹配字符串结尾, 确保整个串长度符合要求

- 重叠匹配 (以markdown中**黑体**, `**黑体**` 为例)

In [55]:
print(re.findall(r"(?=\*\*(.*?)\*\*)", "**font** text**"))

['font', ' text']


- 正常匹配方式 : 每找到一次匹配, 游标移动到这次匹配的末尾, 再从这里找下一个匹配 (不重叠匹配)

  这也是markdown渲染的方式

In [56]:
print(re.findall(r"\*\*(.*?)\*\*", "**font** text**"))

['font']


In [57]:
print(re.findall(r"(?![be])[a-z]", "abcdefgh"))

['a', 'c', 'd', 'f', 'g', 'h']


In [58]:
print(re.findall(r"[a-z](?<![be])", "abcdefgh"))

['a', 'c', 'd', 'f', 'g', 'h']
